In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from shapely.ops import unary_union
import numpy as np
from shapely.strtree import STRtree
import matplotlib.pyplot as plt

In [2]:
gdf = gpd.read_file("../resources/gadm_level2.gpkg")
print(gdf.tail())

/opt/anaconda3/lib/python3.13/site-packages/pyogrio/raw.py:198: RuntimeWarning: This version of GeoPackage user_version=0x000028A0 (10400, v1.4.0) on '../resources/gadm_level2.gpkg' may only be partially supported
  return ogr_read(


           GID_2        NAME_2    GID_1              NAME_1 GID_0    NAME_0  \
48128  ZWE.9.5_2  Gwanda Urban  ZWE.9_1  Matabeleland South   ZWE  Zimbabwe   
48129  ZWE.9.6_2        Insiza  ZWE.9_1  Matabeleland South   ZWE  Zimbabwe   
48130  ZWE.9.7_2        Mangwe  ZWE.9_1  Matabeleland South   ZWE  Zimbabwe   
48131  ZWE.9.8_2        Matobo  ZWE.9_1  Matabeleland South   ZWE  Zimbabwe   
48132  ZWE.9.9_2      Plumtree  ZWE.9_1  Matabeleland South   ZWE  Zimbabwe   

                                                geometry  
48128  MULTIPOLYGON (((29.00187 -20.95896, 29.00164 -...  
48129  MULTIPOLYGON (((29.35744 -21.02481, 29.35821 -...  
48130  MULTIPOLYGON (((27.81436 -21.19368, 27.81452 -...  
48131  MULTIPOLYGON (((28.45053 -21.64326, 28.44812 -...  
48132  MULTIPOLYGON (((27.83201 -20.57665, 27.82436 -...  


In [3]:
df_images = pd.read_csv("../resources/mp16_combined.csv");
print(df_images.head());

                 IMG_ID        AUTHOR        LAT         LON  S3_Label  \
0  92_17_5276763594.jpg  42441750@N03  38.685568 -109.532951       1.0   
1  0d_ce_6392770405.jpg  68149505@N00  34.933793  103.692741       0.0   
2  2a_88_5268406683.jpg  84867026@N00  39.983433  -75.243301       0.0   
3  82_be_2515710583.jpg  75292316@N00  39.306094  -84.379291       1.0   
4  03_05_9498368699.jpg  61068860@N00   9.186625  123.581597       1.0   

   S16_Label  S365_Label   Prob_indoor  Prob_natural  Prob_urban  \
0        7.0       289.0  1.739840e-04      0.897409    0.102417   
1        1.0       122.0  9.968868e-01      0.000578    0.002535   
2        0.0       128.0  7.201538e-01      0.034871    0.244975   
3        6.0       145.0  9.050690e-05      0.516982    0.482927   
4        8.0        36.0  9.902391e-07      0.999983    0.000016   

  neighbourhood          city               county         state  \
0           NaN           NaN         Grand County          Utah   
1         

In [4]:
geometry = [Point(xy) for xy in zip(df_images['LON'], df_images['LAT'])]
gdf_images = gpd.GeoDataFrame(df_images, geometry=geometry, crs="EPSG:4326")

joined = gpd.sjoin(gdf_images, gdf, how='left', predicate='within')

image_counts = joined.groupby(joined.index_right).size()

gdf['image_count'] = gdf.index.map(image_counts).fillna(0).astype(int)

print(gdf[['GID_2', 'NAME_2', 'image_count']].head(20))

   GID_2 NAME_2  image_count
0                        464
1                          0
2                          0
3                          2
4                        523
5                         26
6                        129
7                        729
8                          0
9                         19
10                        22
11                        67
12                      1109
13                        39
14                         1
15                         0
16                        90
17                       287
18                      2590
19                        52


In [11]:
def merge_small_cells_fast(gdf, min_count=50, max_iterations=100):
    """
    Merge small geocells (with < min_count images) into neighboring cells.
    
    Parameters:
    -----------
    gdf : GeoDataFrame
        Input geocells with image_count column
    min_count : int
        Minimum image count threshold (default: 50)
    max_iterations : int
        Maximum number of iterations to prevent infinite loops (default: 100)
    
    Returns:
    --------
    GeoDataFrame with merged geocells
    """
    gdf = gdf.copy()
    
    # Validation: check that image_count exists
    if 'image_count' not in gdf.columns:
        raise ValueError("GeoDataFrame must have 'image_count' column")
    
    # Store original dtypes for proper reconstruction
    col_names = gdf.drop(columns='geometry').columns.tolist()
    dtypes = {col: gdf[col].dtype for col in col_names}
    
    # Keep arrays for speed
    geom = gdf.geometry.values
    counts = gdf["image_count"].values
    countries = gdf["GID_0"].values
    index_map = gdf.index.to_numpy()  # track "alive" indices
    
    # Store all original columns (except geometry) as DataFrame for better column handling
    df_data = gdf.drop(columns='geometry').copy()
    
    count = 0
    total_unmergeable_count = 0  # Track total count of unmergeable cells across all iterations

    while True:
        count += 1
        print(f"Iteration {count}")
        
        small_mask = counts < min_count
        if not small_mask.any():
            print("No more small cells to merge")
            break
            
        if count > max_iterations:
            print(f"Reached maximum iterations ({max_iterations})")
            remaining_small = np.sum(small_mask)
            print(f"Warning: {remaining_small} small cells remain unmerged")
            break

        # Build STRtree only once per iteration (fast!)
        tree = STRtree(geom)
        
        # Mapping: small_id → target_id
        merge_map = {}
        # Mapping: target_id → list of geometry indices to union
        groups = {}
        # Track which cells are targets (to prevent them from being sources)
        protected_targets = set()
        # Track cells that can't find neighbors in this iteration
        unmergeable_cells_this_iter = set()

        small_indices = np.where(small_mask)[0]

        # First pass: collect all merge decisions
        for si in small_indices:
            # Skip if this cell is already a target for another merge
            if si in protected_targets:
                continue
                
            small_geom = geom[si]
            if small_geom is None:
                continue

            # R-tree candidate search
            candidate_ids = tree.query(small_geom)
            candidate_ids = [ci for ci in candidate_ids if ci != si]

            if not candidate_ids:
                unmergeable_cells_this_iter.add(si)
                continue

            # Filter by same country
            same_country = [
                ci for ci in candidate_ids 
                if countries[ci] == countries[si]
            ]
            if not same_country:
                unmergeable_cells_this_iter.add(si)
                continue

            # Filter by intersects (touches unsafe)
            valid = [
                ci for ci in same_country
                if geom[ci].intersects(small_geom)
            ]
            if not valid:
                unmergeable_cells_this_iter.add(si)
                continue

            # Select best neighbor (highest image_count)
            best = max(valid, key=lambda ci: counts[ci])
            
            # Skip if target is also small and will be processed (to avoid conflicts)
            if best in small_indices and best not in protected_targets:
                # Check if target will find a better neighbor
                # For now, allow it but mark as protected
                protected_targets.add(best)

            merge_map[si] = best

            if best not in groups:
                groups[best] = [best]  # start with the target's own geom
            groups[best].append(si)

        if not merge_map:
            print("No valid merges found in this iteration")
            break

        print(f"  Merging {len(merge_map)} cells into {len(groups)} targets")

        # Apply unions and aggregate data
        for target, members in groups.items():
            # Union geometries
            new_geom = unary_union([geom[i] for i in members])
            new_count = sum(counts[i] for i in members)

            # Update geometry and count
            geom[target] = new_geom
            counts[target] = new_count
            
            # Aggregate other columns
            # For string columns: keep target's value (or concatenate if needed)
            # For numeric columns: sum (or average for non-count columns)
            for col in col_names:
                if col == 'image_count':
                    continue  # Already handled
                    
                col_values = [df_data.iloc[i][col] for i in members]
                
                # Handle based on dtype
                if df_data[col].dtype == 'object' or pd.api.types.is_string_dtype(df_data[col]):
                    # For strings, keep target's value (first in members list is target)
                    df_data.iloc[target, df_data.columns.get_loc(col)] = col_values[0]
                elif pd.api.types.is_numeric_dtype(df_data[col]):
                    # For numeric, sum (appropriate for counts) or could use mean
                    if 'count' in col.lower() or 'size' in col.lower():
                        valid_values = [v for v in col_values if pd.notna(v)]
                        if valid_values:
                            df_data.iloc[target, df_data.columns.get_loc(col)] = sum(valid_values)
                        else:
                            df_data.iloc[target, df_data.columns.get_loc(col)] = 0
                    else:
                        # For other numeric, use mean
                        valid_values = [v for v in col_values if pd.notna(v)]
                        if valid_values:
                            df_data.iloc[target, df_data.columns.get_loc(col)] = np.mean(valid_values)
                        else:
                            df_data.iloc[target, df_data.columns.get_loc(col)] = np.nan
                else:
                    # For other types, keep target's value
                    df_data.iloc[target, df_data.columns.get_loc(col)] = col_values[0]

        # Drop merged-away polygons
        # Only drop cells that are in merge_map (i.e., successfully merged)
        # Cells that couldn't find neighbors are NOT in merge_map and should NOT be dropped
        drop_ids = list(merge_map.keys())
        
        # Safety check: ensure unmergeable cells are never dropped
        # This should never happen, but we check as a safeguard
        unmergeable_in_drop = set(drop_ids) & unmergeable_cells_this_iter
        if unmergeable_in_drop:
            print(f"  Warning: Attempted to drop {len(unmergeable_in_drop)} unmergeable cells - removing from drop list")
            drop_ids = [di for di in drop_ids if di not in unmergeable_cells_this_iter]
        
        # Track unmergeable cells count for reporting
        if unmergeable_cells_this_iter:
            total_unmergeable_count += len(unmergeable_cells_this_iter)
            print(f"  {len(unmergeable_cells_this_iter)} cells could not find valid neighbors (kept as-is)")
        
        keep_mask = np.ones(len(geom), dtype=bool)
        keep_mask[drop_ids] = False

        geom = geom[keep_mask]
        counts = counts[keep_mask]
        countries = countries[keep_mask]
        index_map = index_map[keep_mask]
        df_data = df_data.iloc[keep_mask].reset_index(drop=True)

    # Report unmergeable cells
    if total_unmergeable_count > 0:
        print(f"\nSummary: {total_unmergeable_count} cells across all iterations could not find valid neighbors to merge with")
        print("These cells remain in the output as-is (not deleted)")

    # Update image_count in df_data
    df_data['image_count'] = counts

    # Reconstruct final GeoDataFrame with proper dtypes
    # Convert df_data back to dict, preserving dtypes
    data_dict = {}
    for col in col_names:
        data_dict[col] = df_data[col].values
    
    data_dict['geometry'] = geom
    
    out = gpd.GeoDataFrame(data_dict, index=index_map[:len(geom)], crs=gdf.crs)
    
    # Ensure proper dtypes
    for col, dtype in dtypes.items():
        if col in out.columns:
            try:
                out[col] = out[col].astype(dtype)
            except (ValueError, TypeError):
                # If conversion fails, keep as is
                pass

    return out

In [12]:
merged_gdf = merge_small_cells_fast(gdf)

Iteration 1
  Merging 33333 cells into 14915 targets
  506 cells could not find valid neighbors (kept as-is)
Iteration 2
  Merging 6218 cells into 3219 targets
  600 cells could not find valid neighbors (kept as-is)
Iteration 3
  Merging 908 cells into 589 targets
  670 cells could not find valid neighbors (kept as-is)
Iteration 4
  Merging 99 cells into 83 targets
  689 cells could not find valid neighbors (kept as-is)
Iteration 5
  Merging 5 cells into 4 targets
  693 cells could not find valid neighbors (kept as-is)
Iteration 6
No valid merges found in this iteration

Summary: 3158 cells across all iterations could not find valid neighbors to merge with
These cells remain in the output as-is (not deleted)


In [18]:
merged_gdf.to_file("../resources/gadm_merged.gpkg", driver="GPKG")